In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time, random, requests, json, re

In [2]:
s = requests.Session()

# Set headers for the session
s.headers.update({
   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                 "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
   "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
   "Accept-Language": "en-US,en;q=0.9"
})

# Function to fetch a URL with retries
def fetch(url, max_tries=10):
   delay = 3.0
   for i in range(max_tries):
       r = s.get(url, timeout=45)
       if r.status_code in (200, 304):
           # polite delay between successful fetches
           time.sleep(delay + random.random()*2)
           return r
       if r.status_code in (429, 503):  # too many / temporarily blocked
           time.sleep(delay)
           delay *= 2
           continue
       r.raise_for_status()
   raise RuntimeError(f"Failed after {max_tries} tries: {url}")

# Function to extract string from a BeautifulSoup object
def get_text(object, tag, attributes, text=None):
    try:
        return object.find(tag, attrs=attributes, text=text).string.strip()
    except Exception as e:
        return None

# Function for error handling
def maybe(function):
    try:
        return function()
    except:
        return None

In [3]:
base_url = "https://www.lamudi.com.ph"

# Condo For Sale in Metro Manila with Price above 1 peso and Floor Area above 1 sqm
list_url = "https://www.lamudi.com.ph/buy/metro-manila/condo/?foreclosures=excluded&min-price=1&minArea=1&sorting=newest"

start_page = 101
end_page = 120

In [4]:
# Create or clear the output file
with open('lamudi-listings.txt', 'w', encoding="utf-8") as f:
    pass

results = []
batch = []
batch_size = 50

# Fetch listings from the specified pages and write to file
with open('lamudi-listings.txt', 'a', encoding="utf-8") as f:
    for page in range(start_page, end_page + 1):
        response = fetch(list_url + (("&page=" + str(page)) if page > 1 else "" ))
        soup = BeautifulSoup(response.content, "html.parser")
        listings = soup.find_all('div', class_="snippet__content")

        for listing in listings:

            response = fetch(base_url + listing.find('a', href=True)['href'])
            soup = BeautifulSoup(response.content, "html.parser")
            details = soup.find('div', class_="adform__detail")

            # Extract essential details from the listing
            title = get_text(details, 'div', {"class": "main-title"})
            price = get_text(details, 'div', {"class": "prices-and-fees__price"})
            location = get_text(details, 'div', {"class": "view-map__text"})
            bedrooms = get_text(details, 'div', {"class": "details-item-value", "data-test": "bedrooms-value"})
            bathrooms = get_text(details, 'div', {"class": "details-item-value", "data-test": "full-bathrooms-value"})
            property_type = get_text(details, 'span', {"class": "place-features__values", "data-test": "property-type-value"})
            project_name = get_text(details, 'span', {"class": "place-features__values", "data-test": "project-name-value"})
            offer_type = get_text(details, 'span', {"class": "place-features__values", "data-test": "operation-type-value"})
            construction_year = get_text(details, 'span', {"class": "place-features__values", "data-test": "construction-year-value"})
            parking_spaces = get_text(details, 'span', {"class": "place-features__values", "data-test": "parking-spaces-value"})
            ownership_type = get_text(details, 'span', {"class": "place-features__values", "data-test": "ownership-value"})
            floor_area = get_text(details, 'span', {"class": "place-features__values", "data-test": "floor-area-value"})
            floor = get_text(details, 'span', {"class": "place-features__values", "data-test": "floor-value"})
            publish_date = get_text(details, 'div', {"class": "date"})
            condition = get_text(details, 'div', {"class": "facilities__item"}, text=re.compile(r'.*?furnished'))
            project_link = maybe(lambda: details.find('a', {"class": "detail-page-project__link"}, href=True)['href'])

            # Do not include listings with missing essential details
            if price == None or floor_area == None or location == None or project_name == None:
                continue

            record = {
                'title': title,
                'price': price,
                'location': location,
                'full_address': project_name + ", " + location + ", " + "Metro Manila, Philippines",
                'bedrooms': bedrooms,
                'bathrooms': bathrooms,
                'property_type': property_type,
                'project_name': project_name,
                'offer_type': offer_type,
                'construction_year': construction_year,
                'parking_spaces': parking_spaces,
                'ownership_type': ownership_type,
                'floor_area': floor_area,
                'floor': floor,
                'condition': condition,
                'project_link': project_link,
                'publish_date': maybe(lambda: publish_date.split(" - Published by ")[0]),
                'publish_by':  maybe(lambda: publish_date.split(" - Published by ")[1])
            }
            
            # Convert record to JSON line
            json_line = json.dumps(record, ensure_ascii=False)

            # Add to results for DataFrame
            results.append(record)

            # Add to batch for file writing
            batch.append(json_line)

            # Save to file every batch_size items
            if len(batch) == batch_size or page == end_page:
                f.write('\n'.join(batch) + '\n')
                batch = []
            
results

C:\Users\dvpagcaliwagan\AppData\Local\Temp\ipykernel_14408\51325524.py:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  return object.find(tag, attrs=attributes, text=text).string.strip()


[{'title': 'FOR SALE: STUDIO WITH BALCONY IN UPTOWN ARTS BGC TAGUIG CITY',
  'price': '₱ 17,000,000',
  'location': 'Fort Bonifacio, Taguig',
  'full_address': 'Uptown Arts Residence, Fort Bonifacio, Taguig, Metro Manila, Philippines',
  'bedrooms': '1 bedroom',
  'bathrooms': '1 bathroom',
  'property_type': 'Condo',
  'project_name': 'Uptown Arts Residence',
  'offer_type': 'For Sale',
  'construction_year': None,
  'parking_spaces': None,
  'ownership_type': 'Freehold',
  'floor_area': '41 sqm',
  'floor': None,
  'condition': None,
  'project_link': 'https://www.lamudi.com.ph/projects/uptown-arts-residence/',
  'publish_date': '4 hours ago',
  'publish_by': 'Ray Jerico Albano'},
 {'title': 'For Sale: ONE MCKINLEY PLACE BGC Spacious 1 Bedroom with 1 Parking',
  'price': '₱ 16,000,000',
  'location': 'Fort Bonifacio, Taguig',
  'full_address': 'One Mckinley Place, Fort Bonifacio, Taguig, Metro Manila, Philippines',
  'bedrooms': '1 bedroom',
  'bathrooms': '1 bathroom',
  'property_t